### Part II. Neural Networks and Deep Learning

# 10. Introduction to Artificial Neural Networks with Keras

Artificial Neural Networks (ANN) is a Machine Learning model inspired by the networks of biological neurons found in our brains.  

Although they draw from our biological brains, they have slightly evolved to be somehow different.

### Why is it different this time?

ANNs have been around for quite some time, with efforts going back to a seminal paper by McCulloch and Pitts (1943). However, after a long winter started in the 1960s they seem to be back in town as the cool kid. 

Why would this time be different? According to the author:

**1. Data quantity**, which allows ANNs to perform traditional ML on large and complex problems

**2. Computing power**, thanks to Moore's Law, the gaming industry (for GPUs) and cloud computing 

**3. Improved algorithms** (not that different from 1990s, but those differences had huge impact)

**4. Theoretical limitations** (e.g. getting stuck in local optima) are **rather rare** in practice, or **not as serious** as previously thought

**5. Virtual cycle** of applications > reaserch + funding > more and better applications

### Logical Computations with Neurons

_**Note**: Skipping the part on biological neurons_

McCulloch and Pitts proposed a simple model of the biological neuron later known as an **artificial neuron** characterized by one or more binary inputs and one binary output. 

Even with this simple neuron, it is possible to build an ANN that computes any logical proposition:

![ANN](images/10.ANN.png)

Assuming an activation threshold of two, we can see from the picture above how this could work. 

### The Perceptron

The next step in complexity is the _Perceptron_. Invented in 1957 by Frank Rosenblatt, it is based on an artificial neuron called a _threshold logic unit_ (TLU) / _linear threshold unit_ (LTU). Inputs and outputs are numbers and each input is connected with a weight.

TLU computes a weighted sum and then applies a step function to the sum:

1. $z = w_1x_1 + w_2n_2 + \cdots + w_nx_n = x^tw$

2. $h_w(x) = step(z) = step(x^tw)$

![Threshold Logic Unit](images/10.TLU.png)

Two common step functions used in Perceptrons:

1. Heaviside (z) $ = \begin{cases}
0 & z < 0\\
1 & z \ge 0
\end{cases}$

2. Sign (z) $ = \begin{cases}
-1 & z < 0 \\
0 & z = 0 \\
+1 & z > 0
\end{cases}$

A single TLU can be used for linear binary classification (using a threshold, similarly to LogReg or SVM).  
A Perceptron is simply a single layer of TLUs, with each TLU connected to all the inputs. Having multiple output TLU makes possible to perform multioutput classification. 

It is then possible to compute the outputs of a layer of artificial neurons for several instances at once:

$h_{W,b} (X) = \phi (XW + b)$

$X$ = matrix of input features  

$W$ = weights of neurons (expept bias). One row per input neuron and one col for artificial neuron. 

$b$ = vector containing all the connection weights between the bias neuron and the artificial neurons. It has one bias term per artificial neuron.

$\phi$ = activation function. In our case (artificial neuron = TLU), the activation function is a step function.

#### Training

The Perceptron is then trained reinforcing connections that help **reduce the error**. 

More specifically, the Perceptron is fed one training instance at a time, and for each instance it makes its predictions. For every output neuron that produced a wrong prediction, it reinforces the connection weights from the inputs that would have contributed to the correct prediction.

More formally:

$w_{i,y}^{(next step)} = w_{i,y} + \eta (y_j - \hat{y}_j)x_i $ 

* $w_{i,y}$ = connection weight between $i^{th}$ input neuron and $j^{th}$ output neuron

* $x_i$ is the $i^{th}$ input value of the current training instance

* $\hat{y}_j$ is the output of the $j^{th}$ output neuron for the current training instance

* $y_j$ is the target output of the $j^{th}$ output neuron for the current training instance

* $\eta$ is the learning rate

**Note**: the Perceptron decision boundaries are linear, but as long as training instances are linearly separable, it will converge to a solution. 

#### Limitations

The Perceptron is a fairly rudimentary ANN architecture, incapable for example to solve the exclusive or (XOR) classification problem. 

It turns out that some of the limitations of Perceptrons can be eliminated by stacking multiple Perceptrons: the resulting ANN is known as **Multilayer Perceptron (MLP)**. 

### Multilayer Perceptron and Backpropagation

An MLP is composed of multiple layers, generally named *input* (lower) layers, *hidden* layers, *output* (upper) layers. 

**Note**: so far we are only dealing with feedforward neural network (FNN). 

Backpropagation comes into play as a training method for MLP. In short, it computes the the gradient of the network’s error with regard to every single model parameter by going forward and then backwards. It does so many times until it converges. 

**Note**: More specifically, automatically computing gradients is called automatic differentiation, or **autodiff**. The autodiff technique used by backpropagation is called **reverse-mode autodiff**. 

Let's run through the algorithm:

1. **Forward pass**: each mini-batch of training instances is passed to the network’s input layer > hidden layers > output layers

2. **Error measurement**: network error computed using a loss function that compares the desired output and the actual output of the network

3. **Error attribution**: it computes how much each output connection contributed to the error

4. **Reverse pass**: tracing back error from output layer to lower layer down to inputs

5. **Gradient Descent**: tweak all the connection weights in the network, using the error gradients computed earlier

**Note**: it is important to randomize the initial hiddent layers weights (_break the simmetry_). 

In order for this to work, David Rumelhart, Geoffrey Hinton, and Ronald Williams replaced the step function with the logistic (sigmoid) function: 

$\displaystyle \sigma(z) = \frac{1}{1+ e^{(-z)}}$

This was particularly helpful for Gradient Descent, since with a stepwise function there is no gradient to work with. Secondly, having a non-linear function (and a chain of non-linear function over many layers) allows us to theoretically approximate any continuous function.

#### Regression MLPs

We can use MLPs for single variable regression tasks (with a single output neuron) or multivariable regression (with multiple output neurons). 

A typical regression MLP architecture consists of the following **hyperparameters**:
* n. Input neurons
* n. Hidden layers
* n. Neurons per hidden layer
* n. Output neurons
* Hidden activation function
* Output activation function
* Loss activation function

#### Classification MLPs

For a binary classification problem, you just need a single output neuron using the logistic activation function: the output will be a number between 0 and 1, which you can interpret as the estimated probability of the positive class. 

MLP classification **hyperparameters**:
* n. Input neurons
* n. Hidden layers
* n. Output neurons
* Output layer activation (log / softmax)
* Loss function

### Implementing MLPs with Keras

[Keras](https://github.com/keras-team/keras) is a high-level Deep Learning API. On the backend, it relies on one of three popular open source Deep Learning libraries: TensorFlow, Microsoft Cognitive Toolkit (CNTK), and Theano.

**Note**: for simplicity we will use the Tensorflow implementation `tf.keras` (without any specific TF features) therefore altough the code in this chapter can be used on any backend implementation (_usually by changing the imports_).

First thing, let's make sure that [Tensorflow](https://www.tensorflow.org/) is up and running (installing it in a virtualenv is recommended):

In [21]:
import tensorflow as tf

In [22]:
from tensorflow import keras

In [24]:
keras.__version__

'2.2.4-tf'

### Building an Image Classifier Using the Sequential API

Let's use the Fashion MNIST dataset:

In [27]:
# loading dataset
fashion_mnist = keras.datasets.fashion_mnist

In [28]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 7s 2us/step


In [29]:
# check shape
X_train_full.shape

(60000, 28, 28)

In [30]:
# check type
X_train_full.dtype

dtype('uint8')

Let's create a validation set. Also, input scales since we will be working with Gradient Descent:

In [31]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0

In [32]:
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [34]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
"Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [35]:
class_names[y_train[0]]

'Coat'

Time to build our Neural Network!

In [36]:
# create Sequential model (simplest NN)
model = keras.models.Sequential()

In [37]:
# first layer - no pars - preprocessing
model.add(keras.layers.Flatten(input_shape=[28, 28]))

In [38]:
# first Dense hidden layer with 300 neurons
model.add(keras.layers.Dense(300, activation="relu"))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [39]:
# second with 100 neurons
model.add(keras.layers.Dense(100, activation="relu"))

In [40]:
# output layer with 10 neurons - softmax act function since we have exclusive classes
model.add(keras.layers.Dense(10, activation="softmax"))

In [41]:
# model summary (default layer names)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In order to _break simmetry_ all the connection weights have been initialized randomly. We can change the initialization method by setting a specific `kernel_initializer`.  